In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers as opt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses
import pandas as pd
from sklearn.metrics import precision_score,recall_score,f1_score, cohen_kappa_score

In [91]:
DATA_PATH="./Vowel_Data/Formant/"

In [92]:
seed=42

In [93]:
def get_formant(file_path):
    formant=pd.read_csv(file_path,
                delim_whitespace=True,
                header=None,
                names=["time","F1","F2","F3","F4","F5"],
                dtype=np.float64,
                na_values="--undefined--")
    return formant

def get_all_formants(path=DATA_PATH):
    labels=labels=sorted(os.listdir(path))
    all_formant=pd.DataFrame() #empty dataframe to store all_formants
    
    for label_index,label in enumerate(labels):
        folder_formant=pd.DataFrame() #empty dataframe to store formants of indivuals vowels/words i.e. folders

        formant_files=[path + label + '/' + file for file in sorted(os.listdir(path+'/'+label))]
        for file in tqdm(formant_files,"Reading Formants of label -'{}'".format(label)):
            #print(file)
            individual_formant=get_formant(file_path=file)
  
            length=len(individual_formant)
            F_i=individual_formant.loc[2:,'F1':'F5'].values
            F_i1=individual_formant.loc[1:length-2,'F1':'F5'].values
            F_i2=individual_formant.loc[:length-3,'F1':'F5'].values
            assert len(F_i)==len(F_i1)==len(F_i2)
            #individual_formant.loc[2:,['F1_diff2','F2_diff2','F3_diff2','F4_diff2','F5_diff2']]=F_i-2*F_i1+F_i2
            
            individual_formant=individual_formant.dropna()   #drop the first two rows as there are NaN values due to the difference
            
            folder_formant=folder_formant.append(individual_formant,ignore_index=True)
        
        #folder_formant['disp(F5-F1)']=folder_formant['F5']-folder_formant['F1']
        #folder_formant['disp(F4-F3)']=folder_formant['F4']-folder_formant['F3']
        #folder_formant['disp(F5-F3)']=folder_formant['F5']-folder_formant['F3']
        #folder_formant['disp(F5-F4)']=folder_formant['F5']-folder_formant['F4']
        
        folder_formant['label']=label_index  #new columns for encoding label
        
        all_formant=all_formant.append(folder_formant,ignore_index=True)     
    return all_formant

def get_train_test():
    all_formants=get_all_formants()
    
    y=all_formants['label'].values
 
    only_formants=all_formants.drop(labels=['time','label'],axis=1)
    
    standard_formants=(only_formants-only_formants.mean())/only_formants.std()
    
    X=standard_formants.values   

    assert X.shape[0] == len(y)
    return X,y

In [94]:
get_all_formants()

Reading Formants of label -'Formant7': 100%|██████████| 40/40 [00:00<00:00, 206.31it/s]


,time,F1,F2,F3,F4,F5,label
0,0.025,705.97,1456.68,2400.93,3665.61,4788.23,0
1,0.031,698.49,1496.51,2366.53,3721.44,4786.20,0
2,0.038,670.76,1514.62,2428.13,3774.71,4719.93,0
3,0.044,666.49,1524.23,2439.99,3764.21,4651.84,0
4,0.050,667.40,1524.64,2409.82,3699.78,4696.84,0
...,...,...,...,...,...,...,...
14156,0.331,314.53,2646.94,3283.76,3986.25,4475.36,6
14157,0.338,300.73,2592.58,3240.90,4019.52,4490.93,6
14158,0.344,291.53,2523.56,3304.41,3979.67,4419.76,6
14159,0.350,293.56,2454.07,3125.72,3922.58,4395.65,6


In [95]:
X,y=get_train_test()

Reading Formants of label -'Formant7': 100%|██████████| 40/40 [00:00<00:00, 219.51it/s]


In [96]:
kfold=StratifiedKFold(n_splits=4,shuffle=True,random_state=seed)
cv_acc=[]
cv_AUC=[]
cv_f1=[]
cv_kappa=[]

for train,test in kfold.split(X,y):
    y_hot=to_categorical(y)
    
    model_v=Sequential()
    model_v.add(Dense(32, activation='tanh',input_shape=(X[train].shape[1],)))
    #model_v.add(Dense(32, activation='tanh'))
    model_v.add(Dense(16, activation='tanh'))
    model_v.add(Dense(7, activation='softmax'))
    
    model_v.compile(
        optimizer=opt.Adam(learning_rate=0.005),
        loss='categorical_crossentropy',
        metrics=[['accuracy','AUC']]
    )
    model_v.fit(X[train], y_hot[train], epochs=50, batch_size=X[train].shape[0],verbose=0)
    
    scores=model_v.evaluate(X[test],y_hot[test],verbose=0)
    
    print("%s: %.2f" % (model_v.metrics_names[1],scores[1]))
    print("%s: %.2f" % (model_v.metrics_names[2],scores[2]))
    
    yhat_classes = model_v.predict_classes(X[test], verbose=0)
    
    f1 = f1_score(y[test], yhat_classes,average='weighted')
    print('F1: %.2f' % f1)
    
    kappa = cohen_kappa_score(y[test], yhat_classes)
    print('Cohens kappa: %.2f' % kappa)
    
    cv_acc.append(scores[1])
    cv_AUC.append(scores[2])
    cv_f1.append(f1)
    cv_kappa.append(kappa)
    
    print(' ')
    
print("Overall acc: %.2f (+/- %.2f)" % (np.mean(cv_acc),np.std(cv_acc)))
print("Overall AUC: %.2f (+/- %.2f)" % (np.mean(cv_AUC),np.std(cv_AUC)))
print("Overall f1: %.2f (+/- %.2f)" % (np.mean(cv_f1),np.std(cv_f1)))
print("Overall kappa: %.2f (+/- %.2f)" % (np.mean(cv_kappa),np.std(cv_kappa)))

accuracy: 0.61
auc: 0.90
F1: 0.60
Cohens kappa: 0.54
 
accuracy: 0.60
auc: 0.90
F1: 0.59
Cohens kappa: 0.53
 
accuracy: 0.59
auc: 0.90
F1: 0.56
Cohens kappa: 0.51
 
accuracy: 0.60
auc: 0.90
F1: 0.58
Cohens kappa: 0.53
 
Overall acc: 0.60 (+/- 0.01)
Overall AUC: 0.90 (+/- 0.00)
Overall f1: 0.58 (+/- 0.01)
Overall kappa: 0.53 (+/- 0.01)


In [97]:
save_specifier='only_5formants_32_16_tanh_softmax_adam_005_batchfull_seed42'

In [98]:
cv_data_df=pd.DataFrame(data={'acc':cv_acc,
                              'AUC':cv_AUC,
                              'F1':cv_f1,
                              'Kappa':cv_kappa}
                       )
cv_data_df.to_csv('./New_without_CNN/vowel_cv_data_df_'+save_specifier+'.csv')

In [99]:
DATA_PATH="./Word_Data/Formant/"

In [100]:
def get_formant(file_path):
    formant=pd.read_csv(file_path,
                delim_whitespace=True,
                header=None,
                names=["time","F1","F2","F3","F4","F5"],
                dtype=np.float64,
                na_values="--undefined--")
    return formant

def get_all_formants(path=DATA_PATH):
    labels=labels=sorted(os.listdir(path))
    all_formant=pd.DataFrame() #empty dataframe to store all_formants
    
    for label_index,label in enumerate(labels):
        folder_formant=pd.DataFrame() #empty dataframe to store formants of indivuals vowels/words i.e. folders

        formant_files=[path + label + '/' + file for file in sorted(os.listdir(path+'/'+label))]
        for file in tqdm(formant_files,"Reading Formants of label -'{}'".format(label)):
            #print(file)
            individual_formant=get_formant(file_path=file)
  
            length=len(individual_formant)
            F_i=individual_formant.loc[2:,'F1':'F5'].values
            F_i1=individual_formant.loc[1:length-2,'F1':'F5'].values
            F_i2=individual_formant.loc[:length-3,'F1':'F5'].values
            assert len(F_i)==len(F_i1)==len(F_i2)
            #individual_formant.loc[2:,['F1_diff2','F2_diff2','F3_diff2','F4_diff2','F5_diff2']]=F_i-2*F_i1+F_i2
            
            individual_formant=individual_formant.dropna()   #drop the first two rows as there are NaN values due to the difference
            
            folder_formant=folder_formant.append(individual_formant,ignore_index=True)
        
        #folder_formant['disp(F5-F1)']=folder_formant['F5']-folder_formant['F1']
        #folder_formant['disp(F4-F3)']=folder_formant['F4']-folder_formant['F3']
        #folder_formant['disp(F5-F3)']=folder_formant['F5']-folder_formant['F3']
        #folder_formant['disp(F5-F4)']=folder_formant['F5']-folder_formant['F4']
        
        folder_formant['label']=label_index  #new columns for encoding label
        
        all_formant=all_formant.append(folder_formant,ignore_index=True)     
    return all_formant

def get_train_test():
    all_formants=get_all_formants()
    
    y=all_formants['label'].values
 
    only_formants=all_formants.drop(labels=['time','label'],axis=1)
    
    standard_formants=(only_formants-only_formants.mean())/only_formants.std()
    
    X=standard_formants.values   

    assert X.shape[0] == len(y)
    return X,y

In [101]:
get_all_formants()

Reading Formants of label -'Formant7': 100%|██████████| 40/40 [00:00<00:00, 223.08it/s]


,time,F1,F2,F3,F4,F5,label
0,0.025,0.00,247.87,1699.69,2612.09,4240.09,0
1,0.031,0.00,281.54,1867.43,2664.88,4235.19,0
2,0.038,0.00,307.27,2010.72,2676.97,4235.34,0
3,0.044,0.00,312.13,1980.29,2661.43,4244.88,0
4,0.050,0.00,312.40,1804.43,2631.69,4248.25,0
...,...,...,...,...,...,...,...
19682,0.388,426.76,1907.88,2593.99,3717.64,4051.64,6
19683,0.394,427.44,1953.37,2824.18,3329.49,4139.11,6
19684,0.400,420.76,1962.80,2635.39,3325.78,4159.54,6
19685,0.406,380.70,1958.74,2398.14,3586.44,4098.24,6


In [102]:
X,y=get_train_test()

Reading Formants of label -'Formant7': 100%|██████████| 40/40 [00:00<00:00, 238.39it/s]


In [103]:
kfold=StratifiedKFold(n_splits=4,shuffle=True,random_state=seed)
cv_acc=[]
cv_AUC=[]
cv_f1=[]
cv_kappa=[]

for train,test in kfold.split(X,y):
    y_hot=to_categorical(y)
    
    model_w=Sequential()
    model_w.add(Dense(32, activation='tanh',input_shape=(X[train].shape[1],)))
    #model_w.add(Dense(32, activation='tanh'))
    model_w.add(Dense(16, activation='tanh'))
    model_w.add(Dense(7, activation='softmax'))
    
    model_w.compile(
        optimizer=opt.Adam(learning_rate=0.005),
        loss='categorical_crossentropy',
        metrics=[['accuracy','AUC']]
    )
    model_w.fit(X[train], y_hot[train], epochs=50, batch_size=X[train].shape[0],verbose=0)
    
    scores=model_w.evaluate(X[test],y_hot[test],verbose=0)
    
    print("%s: %.2f" % (model_w.metrics_names[1],scores[1]))
    print("%s: %.2f" % (model_w.metrics_names[2],scores[2]))
    
    yhat_classes = model_w.predict_classes(X[test], verbose=0)
    
    f1 = f1_score(y[test], yhat_classes,average='weighted')
    print('F1: %.2f' % f1)
    
    kappa = cohen_kappa_score(y[test], yhat_classes)
    print('Cohens kappa: %.2f' % kappa)
    
    cv_acc.append(scores[1])
    cv_AUC.append(scores[2])
    cv_f1.append(f1)
    cv_kappa.append(kappa)
    
    print(' ')
    
print("Overall acc: %.2f (+/- %.2f)" % (np.mean(cv_acc),np.std(cv_acc)))
print("Overall AUC: %.2f (+/- %.2f)" % (np.mean(cv_AUC),np.std(cv_AUC)))
print("Overall f1: %.2f (+/- %.2f)" % (np.mean(cv_f1),np.std(cv_f1)))
print("Overall kappa: %.2f (+/- %.2f)" % (np.mean(cv_kappa),np.std(cv_kappa)))

accuracy: 0.38
auc: 0.76
F1: 0.33
Cohens kappa: 0.27
 
accuracy: 0.35
auc: 0.76
F1: 0.30
Cohens kappa: 0.24
 
accuracy: 0.36
auc: 0.77
F1: 0.32
Cohens kappa: 0.25
 
accuracy: 0.35
auc: 0.75
F1: 0.30
Cohens kappa: 0.23
 
Overall acc: 0.36 (+/- 0.01)
Overall AUC: 0.76 (+/- 0.00)
Overall f1: 0.31 (+/- 0.01)
Overall kappa: 0.25 (+/- 0.01)


In [104]:
cv_data_df=pd.DataFrame(data={'acc':cv_acc,
                              'AUC':cv_AUC,
                              'F1':cv_f1,
                              'Kappa':cv_kappa}
                       )
cv_data_df.to_csv('./New_without_CNN/Word_cv_data_df_'+save_specifier+'.csv')